# Poor Man's Custom Keyboard

> __Disclaimer__: You're probably better off using a normal macro. This was mainly just for fun.

>If you don't want a macro, just know that you can spend \$10 for a Leonardo with built-in keyboard emulation abilities (no Python required)

With that said, here is the code used to convert serial input into custom keyboard events. The arduino code is included for completness, but the arduino only needs to print some integer to the serial port via `Serial.println(INT);`. If no button is pressed, the arduino should print `-1`

In [ ]:
import serial
import keyboard
import time 

In [2]:
def task_manager():
    keyboard.send("windows")
    time.sleep(.2)
    keyboard.write("task manager")
    time.sleep(.2)
    keyboard.send("enter")

def youtube():
    keyboard.send("windows")
    time.sleep(.2)
    keyboard.write("chrome https://www.youtube.com/feed/subscriptions")
    time.sleep(.2)
    keyboard.send("enter")

In [3]:
codes = {
    "pi":     '\u03C0',
    "theta":  '\u03B8',
    "lambda": '\u03BB',
    "phi":    '\u03D5',
    "degree": '\u00B0',
    "alpha":  '\u03B1',
    "beta":   '\u03B2',
    "alttab": 'alt+tab', # multi-key macro
    "taskmnger": task_manager,
    "youtube": youtube
}

In [4]:
btn_map = {
    0: "alpha", # Serial.println(0);
    1: "beta",  # Serial.println(1);
    2: "degree",# ...
    3: "lambda",
    4: "youtube",
    5: "taskmnger",
}

In [5]:
ser = serial.Serial('COM3', 9600, timeout=10)
go = True
while go:
    try:
        # Wait for serial input
        raw_in = str(ser.readline())[2:-5]

        # Remove junk: b'VALUE\r\n' --> VALUE
        btn = int(raw_in)
        
        if btn != -1:
            # Report keypress to OS
            cmd = codes[btn_map[btn]]

            # multi-step function
            if type(cmd) == type(lambda:0):
                cmd()

            # single Unicode character
            elif len(cmd)==1:
                keyboard.write(cmd)

            # multi-key macro
            else:
                keyboard.send(cmd)

    except KeyError:
        if btn in btn_map:
            print(f"No function mapped to integer {btn}")
        else:
            print(f"Error calling function {btn}")
    except: # exit gracefully
        go = False

else:
    ser.close()